# Speech Feature Extraction using OpenSMILE (GeMapsv01b + ComParE config)

In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
from IPython.display import display
import numpy as np
import os

from typing import List, Optional

# from speech_study.path_conf import loc_data_dir

# opensmile
import opensmile

# tools
import re #stringsplitting
import soundfile as sf #get audio length

In [68]:
# configure user
user = "mitchel"
extracted_feats = False

if user.lower() == "jonas":
    BASE_PATH = Path("/users/jonvdrdo/jonas/data/aaa_contextaware/raw/uz_study/")
elif user.lower() == "mitchel":
#     BASE_PATH = Path("Z:/shares/ghep_lab/2021_VanhollebekeKappen_EEGStudy2_MIST_Cyberball_Audio/")
    BASE_PATH = Path("E:/Data/2020_ResonanceBreathing/")
DATA_PATH = BASE_PATH.joinpath("Data/Raw/Audio")
INTERIM_DATA_PATH = BASE_PATH.joinpath("Data/Interim/Audio")

# Stitch audio files together

Filenames:
HABITUATION: 
1_Habituation-PPTNUM-1Speech1Phase2-1
1_Habituation-PPTNUM-1Speech1Phase3-2

BREATHING:
2_Breathing-PPTNUM-1Speech2Phase2-1
2_Breathing-PPTNUM-1Speech2Phase3-2

CALCULUS:
3_Calculus-PPTNUM-1Speech3Phase2-1
3_Calculus-PPTNUM-1Speech3Phase3-2

SART:
4_SART-PPTNUM-1Speech4Phase2-1
4_SART-PPTNUM-1Speech4Phase3-2

PASSIVE VIEWING:
5_PassiveViewing-PPTNUM-1Speech5Phase2-1
5_PassiveViewing-PPTNUM-1Speech5Phase3-2

OUTPUT_PATH/INTERIM_DATA_PATH:
INTERIM_DATA_PATH = BASE_PATH.joinpath("Data/Interim/Audio")

In [59]:
INTERIM_DATA_PATH = BASE_PATH.joinpath("Data/Interim/Audio")

count = 0
experimentPhases = ['1_Habituation-', '2_Breathing-', '3_Calculus-', '4_SART-', '5_PassiveViewing-']
num = ['1', '2', '3', '4', '5']

for pptnum in tqdm(range(1,201)):
#     for phase in experimentPhases: # For each participant, loop over each audio file
    for phase in range(0,5):
        filename = experimentPhases[phase] + str(pptnum) + '-1Speech' + str(num[phase]) + 'Phase2-1.wav'
#         print(filename)
        if os.path.isfile(DATA_PATH.joinpath(filename)): # Check if file exists, because we don't have all participant numbers
            print('Currently processing: ' + filename)

            # Create input and output filenames
            file1 = str(DATA_PATH.joinpath(filename))
            file2 = str(DATA_PATH.joinpath(experimentPhases[phase] + str(pptnum) + '-1Speech' + str(num[phase]) + 'Phase3-2.wav'))
            outputfile = str(INTERIM_DATA_PATH.joinpath(experimentPhases[phase] + str(pptnum) + '-speech.wav'))

            # create combiner
            cbn = sox.Combiner()
            # create the output file
            cbn.build(
                [file1, file2], outputfile, 'concatenate'
            )

  0%|          | 0/200 [00:00<?, ?it/s]

1_Habituation-1-1Speech1Phase2-1.wav
Currently processing: 1_Habituation-1-1Speech1Phase2-1.wav
2_Breathing-1-1Speech2Phase2-1.wav
Currently processing: 2_Breathing-1-1Speech2Phase2-1.wav
3_Calculus-1-1Speech3Phase2-1.wav
Currently processing: 3_Calculus-1-1Speech3Phase2-1.wav
4_SART-1-1Speech4Phase2-1.wav
Currently processing: 4_SART-1-1Speech4Phase2-1.wav
5_PassiveViewing-1-1Speech5Phase2-1.wav
Currently processing: 5_PassiveViewing-1-1Speech5Phase2-1.wav
1_Habituation-2-1Speech1Phase2-1.wav
Currently processing: 1_Habituation-2-1Speech1Phase2-1.wav
2_Breathing-2-1Speech2Phase2-1.wav
Currently processing: 2_Breathing-2-1Speech2Phase2-1.wav
3_Calculus-2-1Speech3Phase2-1.wav
Currently processing: 3_Calculus-2-1Speech3Phase2-1.wav
4_SART-2-1Speech4Phase2-1.wav
Currently processing: 4_SART-2-1Speech4Phase2-1.wav
5_PassiveViewing-2-1Speech5Phase2-1.wav
Currently processing: 5_PassiveViewing-2-1Speech5Phase2-1.wav
1_Habituation-3-1Speech1Phase2-1.wav
Currently processing: 1_Habituation-3-1

# Extracting features

useful links:
* [opensmile config folder](https://github.com/audeering/opensmile/tree/v3.0.0/config)
* difference between GeMAPS versions [here](https://github.com/audeering/opensmile/blob/v3.0.0/config/gemaps/CHANGES.txt')

**note**: `eGeMAPS` is an _extended_ version of the GeMAPS

feature-level`
* `Functionals`: global segment based features (1 feature per segment)
* `LowLevelDescriptor`: sliding window features (1 feature per window)

In [69]:
# define the feature extraction configs
func_gemaps= opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.Functionals
)

lld_gemaps = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors
)

# we will use the ComParE LLD to calculate frequency-based features on `F0final_sma`
lld_compare = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors
)

In [82]:
if not extracted_feats:
    df_gemaps_func_list: List[pd.DataFrame] = []
    df_gemaps_lld_list: List[pd.DataFrame] = []
    df_compare_lld_list: List[pd.DataFrame] = []

    def _extract_parse_smile_df(s: opensmile.Smile, f: Path) -> pd.DataFrame:
        df_feat = s.process_file(f)
        df_feat = df_feat.reset_index(drop=False)
        df_feat["file"] = df_feat["file"].astype("str")

        df_feat["fileName"] = f.name
        df_feat["fileNum"] = pd.to_numeric(
            df_feat.fileName.map(lambda x: x.split(".")[0].split("_")[-1]),
            errors="coerce",
        )
        
        # Get relevant stuff such as pptID and fileLength
        df_feat["participantNum"] = int(re.findall('\d+', str(f.name))[1]) # Get participant number from file | Always second number in file
        # Check for higher participant numbers if this works else check: https://pythonexamples.org/python-regex-extract-find-all-the-numbers-in-string/ # x = re.findall('[0-9]+', str)
        df_feat["phaseNum"] = int(re.findall('\d+', str(f.name))[0]) # Get phase number from file | Always first number in file
        df_feat["phaseName"] = re.findall('\\D\w+', str(f.name))[0][1:] # Get first instance of a word, but it also takes underscore, so start counting at the second character in string
#         df_feat["recordingPart"] = int(re.findall('\\D\w+', str(f.name))[3][1]) # Get the recording part. So either first minute, or what comes after [1 or 2] # No longer relevant since adding files together
        df_feat["fileLength"] = float(sf.SoundFile(f).frames / sf.SoundFile(f).samplerate) # Let file length so we can average values or something between the two recordings?
        
        return df_feat
    
    counter = 0
    for file in tqdm(list(INTERIM_DATA_PATH.glob("*.wav"))):
        print(file)
        # calculate the global utterance features
        df_gemaps_func_list.append(_extract_parse_smile_df(func_gemaps, f=file))

        # calculate sliding window based utterance features
        df_gemaps_lld_list.append(_extract_parse_smile_df(lld_gemaps, f=file))
        df_compare_lld_list.append(_extract_parse_smile_df(lld_compare, f=file))
        
#         counter = counter + 1
#         if counter == 4:
#             break

    df_gemaps_func = pd.concat(df_gemaps_func_list)
    df_gemaps_lld = pd.concat(df_gemaps_lld_list)
    df_compare_lld = pd.concat(df_compare_lld_list)

    # save the dataframes
    df_gemaps_func.to_parquet(
        INTERIM_DATA_PATH.joinpath("df_gemaps_func.parquet"), engine="fastparquet"
    )
    df_gemaps_lld.to_parquet(
        INTERIM_DATA_PATH.joinpath("df_gemaps_lld.parquet"), engine="fastparquet"
    )
    df_compare_lld.to_parquet(
        INTERIM_DATA_PATH.joinpath("df_compare_lld.parquet"), engine="fastparquet"
    )
    del (
        file,
        df_gemaps_func_list,
        df_gemaps_lld_list,
        df_compare_lld_list,
#         _parse_concat_df,
#         _extract_parse_smile_df,
    )

df_gemaps_func = pd.read_parquet(INTERIM_DATA_PATH.joinpath("df_gemaps_func.parquet"))
df_gemaps_lld = pd.read_parquet(INTERIM_DATA_PATH.joinpath("df_gemaps_lld.parquet"))
df_compare_lld = pd.read_parquet(INTERIM_DATA_PATH.joinpath("df_compare_lld.parquet"))

  0%|          | 0/798 [00:00<?, ?it/s]

E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-1-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-2-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-3-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-4-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-5-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-6-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-7-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-8-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-9-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-13-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-14-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituation-15-speech.wav
E:\Data\2020_ResonanceBreathing\Data\Interim\Audio\1_Habituat

In [81]:
df_gemaps_func

,file,start,end,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,...,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,fileName,fileNum,participantNum,phaseNum,phaseName,recordingPart,fileLength
index,,,,,,,,,,,,,,,,,,,,,
0,D:\Data\2020_ResonanceBreathing\Data\Raw\Audio...,0 days,0 days 00:00:59.980000,30.998972,0.236124,30.352009,33.935448,35.263351,4.911343,489.860107,...,0.234069,0.150307,0.153185,1_Habituation-1-1Speech1Phase2-1.wav,NaN,1,1,Habituation,1,59.980000
0,D:\Data\2020_ResonanceBreathing\Data\Raw\Audio...,0 days,0 days 00:00:03.530929,13.498513,0.082764,12.494122,13.372005,14.075567,1.581446,51.154274,...,0.018397,0.213077,0.173044,1_Habituation-1-1Speech1Phase3-2.wav,NaN,1,1,Habituation,2,3.530930
0,D:\Data\2020_ResonanceBreathing\Data\Raw\Audio...,0 days,0 days 00:00:59.978594,28.413643,0.332545,15.509212,33.827290,35.960094,20.450882,475.313812,...,0.126090,0.149324,0.131786,1_Habituation-9-1Speech1Phase2-1.wav,NaN,9,1,Habituation,1,59.978594
0,D:\Data\2020_ResonanceBreathing\Data\Raw\Audio...,0 days,0 days 00:00:59.978594,29.935358,0.350898,13.989197,34.994003,38.575272,24.586075,430.952332,...,0.121689,0.109820,0.100687,1_Habituation-13-1Speech1Phase2-1.wav,NaN,13,1,Habituation,1,59.978594


In [77]:
float(59.98)

59.98

In [96]:
df_gemaps_func

,file,start,end,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,...,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,fileName,fileNum,participantNum,taskType
0,Z:\shares\ghep_lab\2021_VanhollebekeKappen_EEG...,0 days,0 days 00:01:05.079365,31.011177,0.15535,29.014456,32.646851,33.769093,4.754637,270.609344,...,2.705195,1.84587,0.278,0.220849,0.249231,0.366235,audio_picture_baseline.wav,NaN,10,cybb


## Extract `F0`-range from the LLD's

In [6]:
from tsflex.features import FuncWrapper
from tsflex.features.utils import make_robust
import numpy as np

In [7]:
def quantile_nz(a: np.ndarray, q=List[float]) -> List[Optional[float]]:
    a_nz = a[a>0]
    if len(a_nz):
        return np.quantile(a_nz, q=q)
    else:
        return [None] * len(q)


def nonzero_count(a: np.ndarray) -> int:
    return sum(a > 0)


def return_func_series_list(a: np.ndarray, f_list: List[FuncWrapper]) -> pd.Series:
    s = pd.Series(dtype='float64')
    for f in f_list:
        s = pd.concat([s, pd.Series(data=f(a), index=f.output_names)])
    return s.sort_index()

qs = sum([[1 - q, q] for q in [0, 0.01, 0.02, 0.03, 0.05, 0.1, 0.15, 0.2]], []) + [0.5]
display(str(qs))

'[1, 0, 0.99, 0.01, 0.98, 0.02, 0.97, 0.03, 0.95, 0.05, 0.9, 0.1, 0.85, 0.15, 0.8, 0.2, 0.5]'

### GeMAPS LLD

In [8]:
df_gemaps_lld.filter(like='F0').columns
# logRelF0-H1-H2_sma3nz -> log freq difference between the harmonics
# SMA -> moving average window
# NZ -> no-zero

Index(['F0semitoneFrom27.5Hz_sma3nz', 'logRelF0-H1-H2_sma3nz',
       'logRelF0-H1-A3_sma3nz', 'F1amplitudeLogRelF0_sma3nz',
       'F2amplitudeLogRelF0_sma3nz', 'F3amplitudeLogRelF0_sma3nz'],
      dtype='object')

In [9]:
# define the signal on which the function will be performed, and the functions
s_name = "F0semitoneFrom27.5Hz_sma3nz"

f_gemaps_lld_funcs: List[FuncWrapper] = [
    make_robust(
        FuncWrapper(quantile_nz, output_names=[s_name + f"_q={q}" for q in qs], q=qs),
        min_nb_samples=3,
        passthrough_nans=False,
    ),
    make_robust(FuncWrapper(nonzero_count, output_names=[f"{s_name}_nzcount"])),
]

# Apply the functions on each group
df_gemaps_lld_F0 = (
    df_gemaps_lld.groupby(by=["fileName", "fileNum", "participantNum"])[[s_name]]
    .apply(lambda x: return_func_series_list(x.values, f_list=f_gemaps_lld_funcs))
    .reset_index()
)
display(df_gemaps_lld_F0)

df_gemaps_lld_F0.to_parquet(loc_data_dir.joinpath('df_gemaps_lld_F0.parquet'))

,fileName,fileNum,participantNum,F0semitoneFrom27.5Hz_sma3nz_nzcount,F0semitoneFrom27.5Hz_sma3nz_q=0,F0semitoneFrom27.5Hz_sma3nz_q=0.01,F0semitoneFrom27.5Hz_sma3nz_q=0.02,F0semitoneFrom27.5Hz_sma3nz_q=0.03,F0semitoneFrom27.5Hz_sma3nz_q=0.05,F0semitoneFrom27.5Hz_sma3nz_q=0.1,...,F0semitoneFrom27.5Hz_sma3nz_q=0.2,F0semitoneFrom27.5Hz_sma3nz_q=0.5,F0semitoneFrom27.5Hz_sma3nz_q=0.8,F0semitoneFrom27.5Hz_sma3nz_q=0.85,F0semitoneFrom27.5Hz_sma3nz_q=0.9,F0semitoneFrom27.5Hz_sma3nz_q=0.95,F0semitoneFrom27.5Hz_sma3nz_q=0.97,F0semitoneFrom27.5Hz_sma3nz_q=0.98,F0semitoneFrom27.5Hz_sma3nz_q=0.99,F0semitoneFrom27.5Hz_sma3nz_q=1
0,sentences_occ_0.wav,0.0,2,1277.0,12.435101,13.466747,13.814878,14.208255,15.278906,19.285241,...,23.580828,33.262379,34.514874,34.700854,34.983155,35.451363,35.759081,36.293914,37.114670,37.659714
1,sentences_occ_0.wav,0.0,3,1412.0,12.439996,13.128016,13.414092,13.858814,14.567879,16.099171,...,24.109766,34.545498,35.626716,35.856668,36.069822,36.454732,36.764560,37.275663,37.682383,38.439503
2,sentences_occ_0.wav,0.0,4,1425.0,12.037051,12.537432,12.777922,12.877586,13.102240,13.631784,...,19.587413,23.304682,24.538245,24.755706,25.053704,25.609200,25.926469,26.324248,26.992587,34.139542
3,sentences_occ_0.wav,0.0,5,1460.0,12.260077,13.294396,13.548588,13.964571,14.831079,18.212867,...,32.812402,34.778267,36.406091,36.671796,37.035429,37.782782,37.973423,38.117235,38.503780,39.581223
4,sentences_occ_0.wav,0.0,6,1387.0,12.124546,12.720390,13.350441,13.605247,14.101310,18.546955,...,30.957187,33.413910,36.554290,37.356292,37.810281,38.729762,39.351277,39.905043,40.727717,41.366283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,sentences_occ_8.wav,8.0,79,1611.0,12.252987,12.678028,13.045553,13.255882,13.792040,15.774933,...,23.837996,27.213142,32.003437,32.841087,34.177830,35.795990,36.268011,36.431323,36.936553,37.703159
707,sentences_occ_8.wav,8.0,80,1450.0,12.098284,13.330607,14.518354,15.434285,17.251842,20.492920,...,29.711541,32.349951,34.862546,35.372847,35.982650,37.311396,37.974495,38.444895,39.635105,44.563004
708,sentences_occ_8.wav,8.0,81,1628.0,12.119143,13.010720,13.724793,14.201852,14.453457,17.338896,...,28.003833,35.813665,38.113283,38.762816,39.479161,40.491132,40.955476,41.577183,43.273981,44.185596
709,sentences_occ_8.wav,8.0,82,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ComPaRE LLD

In [10]:
df_compare_lld.filter(like='F0').columns

Index(['F0final_sma'], dtype='object')

In [11]:
s_name = "F0final_sma"

f_compare_lld_funcs: List[FuncWrapper] = [
    make_robust(
        FuncWrapper(quantile_nz, output_names=[s_name + f"_q={q}" for q in qs], q=qs),
        min_nb_samples=3,
        passthrough_nans=False,
    ),
    make_robust(FuncWrapper(nonzero_count, output_names=[f'{s_name}_nzcount']))
]

df_compare_lld_F0 = df_compare_lld.groupby(by=['fileName', 'fileNum', 'participantNum'])[[s_name]].apply(
    lambda x: return_func_series_list(x.values, f_list=f_compare_lld_funcs)
)
display(df_compare_lld_F0)


df_compare_lld_F0.to_parquet(loc_data_dir.joinpath('df_compare_lld_F0.parquet'))

F0final_sma_nzcount  \
fileName            fileNum participantNum                        
sentences_occ_0.wav 0.0     2                            1305.0   
                            3                            1438.0   
                            4                            1484.0   
                            5                            1484.0   
                            6                            1445.0   
...                                                         ...   
sentences_occ_8.wav 8.0     79                           1691.0   
                            80                           1475.0   
                            81                           1654.0   
                            82                              0.0   
                            83                           1588.0   

                                            F0final_sma_q=0  \
fileName            fileNum participantNum                    
sentences_occ_0.wav 0.0     2                     52.270561   
                            3                     52.501904   
                            4                     52.134758   
                            5                     52.093735   
                            6                     52.266987   
...                                                     ...   
sentences_occ_8.wav 8.0     79                    52.626770   
                            80                    52.479759   
                            81                    53.557262   
                            82                          NaN   
                            83                    52.461521   

                                            F0final_sma_q=0.01  \
fileName            fileNum participantNum                       
sentences_occ_0.wav 0.0     2                        55.465402   
                            3                        55.533387   
                            4                        53.612640   
                            5                        54.789709   
                            6                        53.462184   
...                                                        ...   
sentences_occ_8.wav 8.0     79                       53.693527   
                            80                       54.475182   
                            81                       56.566362   
                            82                             NaN   
                            83                       54.255025   

                                            F0final_sma_q=0.02  \
fileName            fileNum participantNum                       
sentences_occ_0.wav 0.0     2                        58.989787   
                            3                        58.160414   
                            4                        54.619430   
                            5                        56.277618   
                            6                        55.384617   
...                                                        ...   
sentences_occ_8.wav 8.0     79                       54.522234   
                            80                       57.765017   
                            81                       58.249159   
                            82                             NaN   
                            83                       56.903896   

                                            F0final_sma_q=0.03  \
fileName            fileNum participantNum                       
sentences_occ_0.wav 0.0     2                        60.276768   
                            3                        59.298107   
                            4                        55.355605   
                            5                        58.652993   
                            6                        56.776181   
...                                                        ...   
sentences_occ_8.wav 8.0     79                       55.599286   
                            80          

In [12]:
del df_compare_lld, df_gemaps_lld

# Join into one big dataframe

In [13]:
df_feat_tot = df_gemaps_func.merge(
    df_gemaps_lld_F0.reset_index(), on=["participantNum", "fileNum", "fileName"]
).merge(df_compare_lld_F0.reset_index(), on=["participantNum", "fileNum", "fileName"])
df_feat_tot

,file,start,end,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,...,F0final_sma_q=0.2,F0final_sma_q=0.5,F0final_sma_q=0.8,F0final_sma_q=0.85,F0final_sma_q=0.9,F0final_sma_q=0.95,F0final_sma_q=0.97,F0final_sma_q=0.98,F0final_sma_q=0.99,F0final_sma_q=1
0,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:22.759541,21.179815,0.180716,18.353466,22.343269,24.074480,5.721014,190.196594,...,65.196510,99.138233,110.025970,112.053701,114.192257,117.051977,119.368555,121.879068,124.523561,142.359802
1,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:22.959541,22.066591,0.160472,21.380507,22.858728,24.399647,3.019140,221.611267,...,91.588757,102.468681,112.185510,114.339508,117.557909,120.699886,123.230260,125.037055,135.241089,185.964005
2,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:22.879541,22.135927,0.156605,21.288734,22.825014,24.613152,3.324417,216.903305,...,90.732471,102.212540,113.653793,116.742743,119.183051,122.307113,125.257780,127.775723,130.264922,149.820251
3,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:23.079541,21.619080,0.194843,20.401165,22.689894,24.232353,3.831188,171.961365,...,70.690057,101.297112,110.718977,114.317660,116.602289,120.677321,123.679845,124.687764,127.257471,132.555847
4,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:24.879541,21.188087,0.204497,16.445494,22.498318,24.085922,7.640429,347.091370,...,61.879944,99.730675,109.675598,111.428383,113.501854,116.884407,119.938060,122.286217,126.239799,150.601013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:24.399541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:22.519541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:21.399541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,/users/jonvdrdo/jonas/data/aaa_contextaware/ra...,00:00:00,00:00:21.519541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_feat_tot.to_parquet(loc_data_dir.joinpath("df_speech_feat_tot.parquet"))